# Manipulation.py: Pulling example


This tutorial provides an example of realizing a "pull" motion of a robot arm on a box using of "manipulation.py" and "robot_execution.py". 
In the context of pulling a box with a robot arm, "pulling" refers to the action of applying force to move the box towards or away from a specific direction or location using the robot arm. This typically involves gripping the box securely with the robot's end-effector and then exerting force to shift the box along a desired path or trajectory. 

As always we start with the necessary imports

In [1]:
import robotic as ry
import manipulation as manip
import numpy as np
import robot_execution as robex

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Now onto a basic configuration with a single blue box

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

midpoint = np.array([-0.105, 0.4, 0.705-.025])
C.addFrame("box") \
    .setPosition(midpoint) \
    .setShape(ry.ST.ssBox, size=[0.04, 0.12, 0.04, 0.001]) \
    .setColor([0, 0, 1]) \
    .setContact(1) \
    .setMass(.1)

# for convenience, a few definitions:
gripper = "l_gripper"
box = "box"
table = "table"

C.view()

0

Now follows the implementation of the pull() function.  The function orchestrates a sequence of actions using a ManipulationModelling instance M to simulate a pulling operation on an object. Two sub-motions, M1 and M2, are derived from M to handle the pulling and approach phases, respectively. 

In [6]:
def pull(object_, info) -> bool:
    M = manip.ManipulationModelling(C, info, ['l_gripper'])
    M.setup_pick_and_place_waypoints(gripper, object_, 1e-1)  # init 2 phase "pick-and-place" problem (in our case applying force and pull)
    M.pull([1.,2.], object_, gripper, table)  # add pull constraint
    placePosition = midpoint + np.random.uniform(-.1, .1, size=3)
    M.komo.addObjective([2.], ry.FS.position, [object_], ry.OT.eq, 1e1*np.array([[1,0,0],[0,1,0]]), placePosition)
    M.solve()

    M1 = M.sub_motion(0, accumulated_collisions=False)
    M1.retractPush([.0, .15], gripper, .03)
    M1.approachPush([.85, 1.], gripper, .03)
    path1 = M1.solve()

    M2 = M.sub_motion(1, accumulated_collisions=False)
    path2 = M2.solve()
    if not M.feasible or not M1.feasible or not M2.feasible:
         return False

    M1.play(C, 1.)
    C.attach(gripper, object_)
    M2.play(C, 1.)
    C.attach(table, object_)


    return Truhandlehandlee

In the case of using a real robot, you can extend this function by incorporating the following commented code segment, which utilizes botop to execute the motion paths generated. It's important to note that the pulling action may not function correctly in simulation due to the challenges associated with accurately simulating physics for this type of movement.

In [14]:
# robot = robex.Robot(C, on_real=True)
# robot.execute_path_blocking(C, path1)
# C.attach(gripper, box)
# robot.execute_path_blocking(C, path2)
# C.attach(table, box)

Now we try a random "pull" motion attempt_count number of times. Try to play around with the placePosition variable or different objects if you like

In [4]:
attempt_count = 100
data = []
for l in range(attempt_count):
    
    action = "pull"
    if action == "pull":
        placePosition = [midpoint[0] + np.random.random()*.4 -.2, midpoint[1] + np.random.random()*.4 -.2]
        object_ = np.random.choice([box])
        success = pull(object_, "")

    else:
        raise Exception(f'Action "{action}" is not defined!')
    
    data.append({"action": action, "success": success})
print(data)

  -- feasible:
     { time: 0.091205, evals: 200, done: 1, feasible: 1, sos: 0.0479341, f: 0, ineq: 0, eq: 0.046638 }
  -- feasible:sub_motion_0--
     { time: 0.012817, evals: 17, done: 1, feasible: 1, sos: 0.5069, f: 0, ineq: 8.11309e-08, eq: 0.000170252 }
  -- feasible:sub_motion_1--
     { time: 0.006128, evals: 7, done: 1, feasible: 1, sos: 0.0213975, f: 0, ineq: 0, eq: 9.59405e-06 }
  -- feasible:
     { time: 0.031937, evals: 83, done: 1, feasible: 1, sos: 0.0336773, f: 0, ineq: 0, eq: 0.0466416 }
  -- feasible:sub_motion_0--
     { time: 0.024041, evals: 37, done: 1, feasible: 1, sos: 0.270484, f: 0, ineq: 1.56589e-07, eq: 3.35213e-05 }
  -- feasible:sub_motion_1--
     { time: 0.003547, evals: 8, done: 1, feasible: 1, sos: 0.0335151, f: 0, ineq: 0, eq: 2.83931e-05 }
  -- feasible:
     { time: 0.016038, evals: 25, done: 1, feasible: 1, sos: 0.0329451, f: 0, ineq: 0, eq: 0.0466644 }
  -- feasible:sub_motion_0--
     { time: 0.026009, evals: 42, done: 1, feasible: 1, sos: 0.2279

KeyboardInterrupt: 